# Notebook for the analysis of tHFA data

## Version control

Changes mades:

- Version 2.7 
    - Changed the record reviewed numbers to back to be based on the number of records as it is not possible to have blank records. This should only be applied for chw interviews and patient interviews 
- Version 2.6
    - Change all the record and interviews to go the 5 and not based on the number of recorded sessions
    - Revised vacancy scores to be nan when both the numerator and denominator are 0
- Version 2.5
    - patient-centeredness: removed the begin criteria so that all 5 patient interviews are checked for their values
    - Updated data for UGA and LBR
- Version 2.4
    - facility composite: Changed TB treatment to treat not documented and client still on treatment as np.nan.
- Version 2.3
    - ISS: Removing places where supervison occurs more than 3 months ago.
    - Gestation age: Changes to gestation age to make 98 (not recorded) return 0
    - Guidelines: Include and condition for guidelines for TB to require both guidelines for children and adults to be present
    - Composite score/DPT: Capped DPT score at 100
    - Included location rows to the pivot table output
- Version 2.2
    - Patient centeredness: Changed the way the no-problem function is applied to be whenever the necessary contains "Other_(Specify)" and not just is equal to it.
    - Patiend centeredness: Included question averaging in the output
    - Patient centeredness: Improved the treatment of nan/'' for questions 'D004' and 'D011'
    - Updated data to remove tertiary and secondary facilities with testing without labs
- Version 2.1
    - Guidelines: Separated the guide_hiv sub-domain to guide-hiv-test and guide_ART
    - Vacancy rate: Capped the score at 100 for places where the unfilled is larger than the funded posts
    - System-CHW: Included number of CHWs per facility
- Version 2.0
    - After verification of data with TGF
- Version 1.4
    - High Quality Services:
        - Patient-centeredness: changed the way that challenges were processed to include the list of responses that are not challenges
- Version 1.3 (Aug 9, 2024)
    - High Quality Services:
        - Patient-centeredness: Averaged the responses per client first and then averaged client scores for facilities
    - ISS indicator:  
        - technical content: Addressed error on this indicator. This error was based on a faulty estimation of the conduct of HTM by facilities. A faulty estimation that used the sum of ART, HIV-testing, TB, and malaria was used.
        - Summary Stats: Treated "DNK" as NaN instead of 0
    - CHW indicator: 
        - ISS: to specify that if HTM is conducted in a session then it is ISS and if CHW doesn't support HTM with at least 2 diseases results in ISS. 
        - contract and tools: "DNK" are treated as "No" as an interim step from TGF (As per conversation with Pankhuri on Aug 9, 2024)
    - Oxygen servcies: Indicator to only use "Observed" as 100 (Remove "Reported, Not seen") 
    - Lab testing: Indicator to count "DNK" as "No"
- Version 1.2 (Aug 5, 2024)
    - Fixed the problem with the ISS for CHWs having all 100. The formula was wrong and did not actually compare expected service areas with ISS service areas per month
    - There was a fix for the implementation of any(). This fix also caused small changes in the group-PS dimension of the iSS indicator.
- Version 1.1
    - Updated the outputs to include the underlying attributes
    - Used the QCed excel files provided by Devashish
    - Included a function to make the necessary pivot tables


- Version 1.0
    - Version used to start writing the report for tHFA in Malawi
    - Version shared with TGF for further comparison on Wed July 31
- Version 0.9
    - Updated oxygen function to correct formulas based on the tHFA Annex 1
    - Updated guidelines function to correct formula (there was a problem with using "value == target1 or target2")
    - Minor changes to facility score indicator
    - Changed high quality header name to "Quality_Score"
- Version 0.8
    - Included testing capacity indicator
- Version 0.7
    - Included all indicators except testing capacity
    - Sent to TGF for comparison


## Data loading

In [75]:
# input cells
code = ''
version = '2.7'

In [76]:
# file creation
countries = {
    'GHA': 'ghana',
    'GMB': 'gambia',
    'IDN': 'indonesia',
    'LBR': 'liberia',
    'MDG': 'madagascar',
    'MWI': 'malawi',
    'NGA': 'nigeria',
    'UGA': 'uganda',
    'COG': 'congo',
}

country = countries[code]

import os
os.makedirs(f'./output/{code}/',exist_ok=True)

file_path = f"./Updated tHFA data/tHFA_{code}.xlsx"

pivot_out = f"./output/{code}/{country}_pivot_v{version}.csv"
indicators_out = f'./output/{code}/tHFA-{country}-analysis_v{version}.csv'
map_out = f"./output/{code}/{code}-facility-points.csv"

In [77]:
# package import and data loading
import pandas as pd
import numpy as np
import math

# Step 1: Load the CSV file into a pandas DataFrame

df = pd.read_excel(file_path, header=None)
metadata = dict(zip(df.iloc[1], df.iloc[0]))
df.columns = df.iloc[0]
df = df[1:]
df.set_index('Q100', inplace=True)
survey_details_df = df.iloc[:, :11]
facility_details_df = pd.concat([df.iloc[:, 11:23], df.iloc[:, 29:37]], axis=1)
facility_details_df.to_csv(map_out)
facility_details_df = facility_details_df.drop(columns=['FI_2','FI_3',"Q100a",'Q106_a','Q109-Latitude','Q109-Longitude','Q109-Altitude', 'Q109-Accuracy'])


test = df[:5]
test_fac_df = facility_details_df[:5]


## Indicator and service functions

### KPI Indicators

In [78]:
# Processing of no problems
file_name = "./reasons/no-problem-reasons_all.txt"
with open(file_name, 'r') as file:
    no_problems = file.read().splitlines()

def problem_check(reason):
    """
    Function to check if a given reason is in the no_problems list.
    
    Parameters:
    reason (str): The reason to check.
    
    Returns:
    int: 100 if the reason is in no_problems, 0 otherwise.
    """
    # Ensure the input reason is treated as a string
    see = str(reason)
    
    # Convert the string to lowercase
    see = see.lower()
    
    # Check if the lowercase string is in the no_problems list
    answer = 100 if see in no_problems else 0
    return answer



In [79]:
# Function for high quality service
def calculate_service_quality(row):
    
    """

   Description


    """

    provides_ART = 1 if row['R2312'] == 'YES' else 0
    provides_TB = 1 if row['R2400'] == 'YES' else 0
    provides_ANC = 1 if row['R1810'] == 'YES' else 0



    # Calculate HW competence scores
    # ANC scores
    hw_scores = []
    anc_scores = []
    tb_scores = []
    art_scores = []

    if provides_ANC > 0:
        anc_count = sum (1 for col in [f'RR_ANC_Begin_{a}' for a in range(1,6)] if row[col] == 'YES')
        
        for i in range(1, anc_count + 1):
            client_anc_scores = []
            # Attribute 1: HIV testing and counselling
            if row[f'Q13009_1_{i}'] == 'YES':
                client_anc_scores.append(np.nan)
            elif row[f'tHFA_F003_{i}'] == 'YES':
                client_anc_scores.append(100)
            elif row[f'tHFA_F003_{i}'] == 'NO':
                client_anc_scores.append(0)
            else:
                client_anc_scores.append(np.nan)

            # Attribute 2: Access to ART
            if row[f'tHFA_F001_{i}'] == 'NO':
                client_anc_scores.append(np.nan)
            elif row[f'tHFA_F001_{i}'] == 'YES' and row[f'tHFA_F004_{i}'] == 'YES':
                client_anc_scores.append(100)
            elif row[f'tHFA_F001_{i}'] == 'YES' and row[f'tHFA_F004_{i}'] == 'NO':
                client_anc_scores.append(0)
            else:
                client_anc_scores.append(np.nan)

            # Attribute 3: Access to iptp
            if row[f'Q13007_3_{i}'] == 'YES':
                client_anc_scores.append(100)
            elif row[f'Q13007_3_{i}'] == 'NO':
                client_anc_scores.append(0)
            else: 
                client_anc_scores.append(np.nan)

            # Attribute 4: Access to TB screening
            tb_values = [row[f'TBSF002_A_{i}'],
                         row[f'TBSF002_B_{i}'], 
                         row[f'TBSF002_C_{i}'],
                         row[f'TBSF002_D_{i}']]
            if all(val == 'YES' for val in tb_values):
                client_anc_scores.append(100)
            else:
                client_anc_scores.append(0)
            
            # Attritube 5, 7, 8, 9, 10, 11
            # Integrated ANC and SRH, Blood pressure, Folic acid, Danger signs
            # Intestinal worms, and hemoglobin
            q = ['13005_1',
                    '13004_8',
                    '13004_10',
                    '13004_11',
                    '13005_2',
                    '13005_5']
            
            for attr in q: 
                if row[f'Q{attr}_{i}'] == 'YES':
                    client_anc_scores.append(100)
                elif row[f'Q{attr}_{i}'] == 'NO':
                    client_anc_scores.append(0)
                else: 
                    client_anc_scores.append(np.nan)

            # Attribute 6: ANC age
            # if row[f'Q13004_1_{i}']
            if int(row[f'Q13004_1_{i}']) == 98: 
                client_anc_scores.append(0)
            elif int(row[f'Q13004_1_{i}']) >= 32:
                client_anc_scores.append(100)
            elif int(row[f'Q13004_1_{i}']) < 32:
                client_anc_scores.append(0)
            else: 
                client_anc_scores.append(np.nan)
            
            client_anc_score = np.nanmean(client_anc_scores)
            anc_scores.append(client_anc_score)

    if provides_ART > 0:
        art_count = sum (1 for col in [f'RR_ART_Begin_{a}' for a in range(1,6)] if row[col] == 'YES')
        
        for i in range(1, art_count+1):
            client_art_scores = []
            # Attribute 1: TB screening
            tb_values = [row[f'Q13408_01_{i}'],
                        row[f'Q13408_02_{i}'], 
                        row[f'Q13408_03_{i}'],
                        row[f'tHFAG003_{i}']]
            if all(val == 'YES' for val in tb_values):
                client_art_scores.append(100)
            else:
                client_art_scores.append(0)
            
            #Attribute 2 : TB treatment for ART patients
            if row[f'Q13408_08_{i}'] == 'ACTIVE TB':
                client_art_scores.append(100) if row[f'Q13408_09_{i}'] == 'YES' else client_art_scores.append(0) 
            else: client_art_scores.append(np.nan)

            # Attribute 3: BP measured
            bp = 'tHFAG001'
            if row[f'{bp}_{i}'] == 'YES':
                client_art_scores.append(100)
            elif row[f'{bp}_{i}'] == 'NO':
                client_art_scores.append(0)
            else: 
                client_art_scores.append(np.nan)
            
            client_art_score = np.nanmean(client_art_scores)
            art_scores.append(client_art_score)
        
    if provides_TB >0 :
        tb_count = sum (1 for col in [f'RR_TB_Begin_{a}' for a in range(1,6)] if row[col] == 'YES')

        for i in range(1,tb_count+1):
            client_tb_scores=[]

            # Attribute 1 and 2:
            # HIV test and symptpms
            tb = ['Q13507_01',
                'Q13510_01']
        
            for attr in tb: 
                if row[f'{attr}_{i}'] == 'YES':
                    client_tb_scores.append(100)
                elif row[f'{attr}_{i}'] == 'NO':
                    client_tb_scores.append(0)
                else: 
                    client_tb_scores.append(np.nan)
            
            client_tb_score = np.nanmean(client_tb_scores)
            tb_scores.append(client_tb_score)
    
    
    anc_score = np.nanmean(anc_scores) if len(anc_scores) > 0 else np.nan
    art_score = np.nanmean(art_scores) if len(art_scores) > 0 else np.nan
    tb_score = np.nanmean(tb_scores) if len(tb_scores) > 0 else np.nan
    hw_scores = anc_scores + art_scores + tb_scores
    hw_score = np.nanmean(hw_scores) if len(hw_scores) > 0 else np.nan
   

    def calculate_column_score(value, column):
        if column in ['tHFA_D004']:
            if (value == 'YES' and row[f'tHFA_D005_{i}'] != "DON’T KNOW"):
                return 100  
            elif value == "NO":
                return 0
            elif (value == 'YES' and row[f'tHFA_D005_{i}'] == "DON’T KNOW"):
                return 0
            elif value == "DON’T KNOW":
                return np.nan
            elif np.isnan(float(value)):
                return np.nan
            else: return np.nan
        elif column == 'tHFA_D006':
            if value == 'YES':  return 0 
            elif value == 'NO': return 100
            else: return np.nan
        elif column == 'tHFA_D011':
            value = str(value)
            if 'OTHERS_(SPECIFY)' in value:
                return problem_check(row[f'tHFA_D011_1_{i}'])
            elif value == "DON'T_KNOW" or value == 'nan':
                return np.nan
            else: return 0
        elif column in ['PREQ132', 'PREQ122', 'PREQ125', 'PREQ119', 'PREQ129']:
            scale = {'NEVER': 0, 'RARELY': 25, 'SOMETIMES': 50, 'OFTEN': 75, 'ALWAYS': 100}
            return scale.get(value, np.nan)
        else:
            if value == 'YES':
                return 100 
            elif value == 'NO':
                return 0
            else: return np.nan

    # Calculate Patient centeredness scores
    patient_centeredness_columns = [
        'PREQ132', # Communication
        'PREQ122', # Respect
        'PREQ125', # Autonomy
        'PREQ119', # Confidentiality
        'PREQ129', # Social support
        'tHFA_D003', # HW told
        'tHFA_D004', # Danger signs
        'tHFA_D006', # Rudeness
        'tHFA_D011' # Challenges
    ]
    patient_count = sum(1 for col in [f'PE_Begin_{i}' for i in range(1,6)] if row[col] == 'YES')
    pc_scores = []
    pc_scores_attr = []
    
    # for i in range(1, patient_count + 1):
    for i in range(1, 6):
        pc_i_scores = []
        for col in patient_centeredness_columns:
            interview_col = f'{col}_{i}'
            if interview_col in row:
                attr_score = calculate_column_score(row[interview_col], col)
                pc_i_scores.append(attr_score)
                pc_scores_attr.append(attr_score)

    
        pc_i_score = np.nanmean(pc_i_scores) if len(pc_i_scores) > 0 else np.nan
        pc_scores.append(pc_i_score)
    
    pc_score = np.nanmean(pc_scores) if len(pc_scores) > 0 else np.nan
    
    # Calculate facility score
    facility_score = np.nanmean([hw_score, pc_score])

    pc_attr_avg = {}
    ques = [
            'Communication',
            'Respect',
            'Autonomy',
            'Confidentiality' ,
            'Social support',
            'HW told',
            'Danger signs',
            'Rudeness',
            'Challenges',
    ]

    for i in range(9):
        indexes =  [j for j in range(i,len(pc_scores_attr),9)]
        values = [pc_scores_attr[idx] for idx in indexes]
        pc_attr_avg[f"PC_avg_{ques[i]}"] = np.nanmean(values)

    result = {
        'Quality_Score': facility_score,
        'HW_Competence_Score': hw_score,
        'HW_Check': hw_scores,
        'ANC_score': anc_score,
        'ART_score': art_score,
        'TB_score': tb_score,
        'Patient_Centeredness_Score': pc_score,
        'PC_Individuals': pc_scores,
        'PC_attributes': pc_scores_attr        
    }

    result.update(pc_attr_avg)

    return pd.Series(result)

    
# test = test.append(df.loc[['COG_000100',
# 'COG_000121',
# 'COG_000268']])
# see = test.apply(calculate_service_quality, axis=1)
# see


In [80]:
# Function for calculation of integrated HTM for ANC clients 

def calculate_HTM_ANC(row):
    
    """

    Pregnant women receive the following services in an integrated fashion: 
    HIV counseling and testing, access to ARVs for HIV positive women, IPTp, 
    and TB screening.


    """

    provides_ART = 1 if row['R2312'] == 'YES' else 0
    provides_TB = 1 if row['R2400'] == 'YES' else 0
    provides_malaria = 1 if row['R1400'] == 'YES' else 0
    provides_hiv_test = 1 if row['R2300'] == 'YES' else 0
    provides_anc = 1 if row['R1810'] == 'YES' else 0
    provides_testing = 1 if row['R2900'] == 'YES' else 0
    provides_oxygen = 1 if row["R1323"] == 'YES' else 0

       
    htm_anc_scores = []
    hiv_testing_scores = []
    ART_access_scores = []
    iptp_access_scores = []
    TB_screening_scores = []
    if provides_anc > 0:
        anc_count = sum(1 for col in ['RR_ANC_Begin_1', 'RR_ANC_Begin_2', 'RR_ANC_Begin_3', 'RR_ANC_Begin_4', 'RR_ANC_Begin_5'] if row[col] == 'YES')
        for i in range(1, anc_count+1):            
            client_anc_scores = []
            # Attribute 1: HIV testing and counselling
            if row[f'Q13009_1_{i}'] == 'YES':
                client_anc_scores.append(np.nan)
            elif row[f'tHFA_F003_{i}'] == 'YES':
                client_anc_scores.append(100)
            elif row[f'tHFA_F003_{i}'] == 'NO':
                client_anc_scores.append(0)
            else:
                client_anc_scores.append(np.nan)
            
            hiv_testing_scores.append(client_anc_scores[-1])

            # Attribute 2: Access to ART
            if row[f'tHFA_F001_{i}'] == 'NO':
                client_anc_scores.append(np.nan)
            elif row[f'tHFA_F001_{i}'] == 'YES' and row[f'tHFA_F004_{i}'] == 'YES':
                client_anc_scores.append(100)
            elif row[f'tHFA_F001_{i}'] == 'YES' and row[f'tHFA_F004_{i}'] == 'NO':
                client_anc_scores.append(0)
            else:
                client_anc_scores.append(np.nan)
            ART_access_scores.append(client_anc_scores[-1])

            # Attribute 3: Access to iptp
            if row[f'Q13007_3_{i}'] == 'YES':
                client_anc_scores.append(100)
            elif row[f'Q13007_3_{i}'] == 'NO':
                client_anc_scores.append(0)
            else: 
                client_anc_scores.append(np.nan)
            iptp_access_scores.append(client_anc_scores[-1])

            # Attribute 4: Access to TB screening
            tb_values = [row[f'TBSF002_A_{i}'],
                         row[f'TBSF002_B_{i}'], 
                         row[f'TBSF002_C_{i}'],
                         row[f'TBSF002_D_{i}']]
            if all(val == 'YES' for val in tb_values):
                client_anc_scores.append(100)
            else:
                client_anc_scores.append(0)
            TB_screening_scores.append(client_anc_scores[-1])

            client_anc_score = np.nanmean(client_anc_scores)
            htm_anc_scores.append(client_anc_score)
        

        htm_anc_score = np.nanmean(htm_anc_scores)

    else:
        htm_anc_score = np.nan

    hiv_testing_score = np.nanmean(hiv_testing_scores) if len(hiv_testing_scores)>0 else np.nan 
    ART_access_score = np.nanmean(ART_access_scores) if len(ART_access_scores)>0 else np.nan 
    iptp_access_score = np.nanmean(iptp_access_scores) if len(iptp_access_scores)>0 else np.nan 
    TB_screening_score = np.nanmean(TB_screening_scores) if len(TB_screening_scores)>0 else np.nan 

    return pd.Series({
        'HTM-ANC': htm_anc_score,
        # 'htm-anc-check': htm_anc_scores,
        'hiv testing': hiv_testing_score,
        'ART access': ART_access_score,
        'iptp scores': iptp_access_score,
        'TB screening': TB_screening_score
        })

# see = test.apply(calculate_HTM_ANC, axis=1)
# see

In [81]:
# Function to calculate system readiness

def calculate_system_CHW(row):
    
    """

   KPI S5: “Systems readiness for CHWs”


    """

    chw_count = sum(1 for col in ['CHW_begin_1', 'CHW_begin_2', 'CHW_begin_3', 'CHW_begin_4', 'CHW_begin_5'] if row[col] == 'YES')
    
    # add consent E002_1
    paid_columns = ['tHFA_E020','tHFA_E021', 'tHFA_E022', 'tHFA_E023']

    chw_scores = []

    chw_atttributes = {
        	'fac_supvsn_scores': [],
            'fac_contract_scores': [],
            'fac_paid_scores': [],
            'fac_jobtools_scores': [],
    }

    for i in range(1, 6):
        chw_i_score = np.nan
        chw_i_scores = []

        if row[f'tHFA_E002_{i}'] != 'YES':
            continue

        # Attribute 1 (ISS)
        expected = set([f'{b}' for b in ['A','B', 'C', 'D','E'] if row[f'tHFA_E003_{b}_{i}'] == 'YES'])
        month_1 = set([f'{b}' for b in ['A','B', 'C', 'D','E'] if row[f'tHFA_E005_{b}_{i}'] == 'YES'])
        month_2 = set([f'{b}' for b in ['A','B', 'C', 'D','E'] if row[f'tHFA_E007_{b}_{i}'] == 'YES'])
        month_3 = set([f'{b}' for b in ['A','B', 'C', 'D','E'] if row[f'tHFA_E009_{b}_{i}'] == 'YES'])
        htm = set(['A','B','C'])
        if htm.issubset(expected):
            chw_iss = 100 if any([htm.issubset(month_1),htm.issubset(month_2),htm.issubset(month_3)]) else 0
        else:
            chw_iss = 100 if any([len(expected.intersection(month_1))>= 2, len(expected.intersection(month_2))>= 2, len(expected.intersection(month_3))>= 2 ]) else 0
        chw_i_scores.append(chw_iss)
        chw_atttributes['fac_supvsn_scores'].append(chw_iss)

        # Attribute 2 (Presence of contract)
        contract = [f'tHFA_E0{b}_{i}' for b in range(10,20)]
        contract_scores = []
        for question in contract:
            if row[question]=='YES': contract_scores.append(100)
            # elif row[question]=='NO': contract_scores.append(0) # To treat 'DNK' as 0 
            else: contract_scores.append(0) # To treat 'DNK' as 0
        chw_contract = 100 if np.nanmean(contract_scores) == 100 else 0
        chw_i_scores.append(chw_contract)
        chw_atttributes['fac_contract_scores'].append(chw_contract)
        
        # Attribute 3 (Payment accoording to contract)
        paid_score = 0 
        for col in paid_columns:
            interview_col = f'{col}_{i}'
            if row[interview_col] == 'YES': 
                paid_score = paid_score + 1
        chw_paid = 100 if paid_score == 4 else 0  
        chw_i_scores.append(chw_paid)
        chw_atttributes['fac_paid_scores'].append(chw_paid)

        # Attribute 4 (Job tools)
        tools = [f'tHFA_E0{b}_{i}' for b in [25,27,29]]
        tools_scores = []
        for question in tools:
            if row[question]=='YES': tools_scores.append(100)
            # elif row[question]=='NO': tools_scores.append(0) # To treat 'DNK' as 0 
            else: tools_scores.append(0) # To treat 'DNK' as 0 
        chw_tools = 100 if np.nanmean(tools_scores) == 100 else 0
        chw_i_scores.append(chw_tools)
        chw_atttributes['fac_jobtools_scores'].append(chw_tools)

        chw_i_score = np.nanmean(chw_i_scores)
        chw_scores.append(chw_i_score)

    chw_score = np.nanmean(chw_scores)
    
    chw_means = {}

    # Loop through the dictionary and calculate the mean for each score list
    for key, value in chw_atttributes.items():
        mean_key = key.replace('_scores', '_score')  # Create a new key for the mean
        chw_means[mean_key] = np.nanmean(value)


    return pd.Series({
        'system_CHW_score': chw_score,
        'chw_count': chw_count,
        'chw_show': chw_scores,
        'chw_iss': chw_means['fac_supvsn_score'],
        'chw_contract': chw_means['fac_contract_score'],
        'chw_paid': chw_means['fac_paid_score'],
        'chw_tools': chw_means['fac_jobtools_score'],
        # 'chw_iss_deep': chw_atttributes['fac_supvsn_scores']

        })

# see = test.apply(calculate_system_CHW, axis=1)
# see

In [82]:
# Function for integrated supportive supervision

def calculate_ISS(row):
    
    """

   Description


    """

    provides_ART = 1 if row['R2312'] == 'YES' else 0
    provides_TB = 1 if row['R2400'] == 'YES' else 0
    provides_immu = 1 if row['R2100'] == 'YES' else 0
    provides_imci = 1 if row['R2000'] == 'YES' else 0
    provides_malaria = 1 if row['R1400'] == 'YES' else 0
    provides_hiv_test = 1 if row['R2300'] == 'YES' else 0
    provides_anc = 1 if row['R1810'] == 'YES' else 0
    provides_testing = 1 if row['R2900'] == 'YES' else 0
    provides_oxygen = 1 if row["R1323"] == 'YES' else 0
    provides_hiv = 1 if provides_hiv_test + provides_ART > 0 else 0

    htm = [provides_hiv,provides_TB,provides_malaria]
    all = [provides_hiv,provides_TB,provides_malaria,provides_anc,provides_imci,provides_immu]
    fac_tech =[]
    fac_stats = []
    fac_group_PS = []
    fac_comms_data = []
    iss_scores = []
    
    # Attribute 1 (Supervision in the past 3 months)
    if row['M610'] == 'NO':
        iss_score = 0
        fac_iss = 0

    elif row['M610'] == 'YES'and row ['M611'] in ['4-12 MONTHS AGO', 'MORE THAN 12 MONTHS AGO']:
        iss_score = 0
        fac_iss = 0

    elif row['M610'] == 'YES'and row ['M611'] in ['DON’T KNOW', 'nan',]:
        iss_score = np.nan
        fac_iss = np.nan
        

    elif row['M610'] == 'YES' and row ['M611'] in ['WITHIN THE PAST MONTH',  '2-3 MONTHS AGO']:
        fac_iss = 100
        
        # Attribute 2 (Technical content)
        if sum(htm) == 3:
            iss_done  = sum(1 if any([row[f'tHFA_C00{i}_{j}'] == 'YES' for i in range(1, 7)]) else 0 for j in ['A', 'B', 'C'])
            iss_scores.append(100) if iss_done == 3 else iss_scores.append(0)
        elif sum(all) >= 3:
            iss_done = sum(1 if any([row[f'tHFA_C00{i}_{j}'] == 'YES' for i in range(1, 7)]) else 0 for j in ['A', 'B', 'C','D','E','F'])
            iss_scores.append(100) if iss_done >= 3 else iss_scores.append(0)
        else:
            iss_scores.append(np.nan)
        fac_tech.append(iss_scores[-1])
        

        # Attribute 3 (Summary stats discussed)
        if row['tHFA_C007'] == 'YES':
            iss_scores.append(100)
        elif row['tHFA_C007'] == 'DON’T KNOW':
            iss_scores.append(np.nan) 
        else: iss_scores.append(0)
        fac_stats.append(iss_scores[-1])
    
        # Attribute 4 (Group problem solving)
        if row['tHFA_C008'] == 'YES' and row['tHFA_C009'] == 'YES':
            iss_scores.append(100)
        elif any([row['tHFA_C008'] == "DON’T KNOW", row['tHFA_C009'] == "DON’T KNOW"]):
            iss_scores.append(np.nan)
        else: 
            iss_scores.append(0)

        fac_group_PS.append(iss_scores[-1])

        # Attribute 5 (Data on community activities)
        if row['tHFA_C010'] == 'NO':
            iss_scores.append(np.nan)
        elif row['tHFA_C010'] == 'YES' and row['tHFA_C011'] == 'DON’T KNOW':
            iss_scores.append(np.nan)
        elif row['tHFA_C010'] == 'YES' and row['tHFA_C011'] == 'NO':
            iss_scores.append(0)
        elif row['tHFA_C010'] == 'YES' and row['tHFA_C011'] == 'YES':
            iss_scores.append(100)
        else: 
            iss_scores.append(np.nan)
        
        fac_comms_data.append(iss_scores[-1])

        iss_score = np.nanmean(iss_scores)
    else: 
        iss_score = iss_scores = fac_iss = np.nan 
        fac_tech = np.nan
        fac_stats = np.nan
        fac_comms_data = np.nan
        fac_group_PS = np.nan
        

    return pd.Series({
        'iss_score': iss_score,
        'iss_check': iss_scores,
        'fac_iss': fac_iss,
        'fac_tech': np.nanmean(fac_tech),
        'fac_stats': np.nanmean(fac_stats),
        'fac_group_PS': np.nanmean(fac_group_PS),
        'fac_comms_data': np.nanmean(fac_comms_data),
        
        })

# test = test.append(df.loc['MWI_000491'])
# see = test.apply(calculate_ISS, axis=1)
# see

### Non-KPI indicators

In [83]:
# Function for calculation of facility composite indicator 

def calculate_facility_composite(row):
    
    """

    RSSH/PP HRH-6: Percentage of facilities providing effective services

    Composite facility level indicator with seven components: 
    1)	% of facilities observed to provide integrated services at ANC (TB, malaria, HIV) at the time of visit; 
    2)	Provider availability (absence rate on day of visit);
    3)	Provider caseload (number of outpatient visits per clinician per day);
    4)	ANC dropout rate; 
    5)	DPT dropout rate; 
    6)	Treatment completion rate for new TB cases;
    7)	Twelve-month retention on ART


    """

    provides_ART = 1 if row['R2312'] == 'YES' else 0
    provides_TB = 1 if row['R2400'] == 'YES' else 0
    provides_immu = 1 if row['R2100'] == 'YES' else 0
    provides_imci = 1 if row['R2000'] == 'YES' else 0
    provides_malaria = 1 if row['R1400'] == 'YES' else 0
    provides_hiv_test = 1 if row['R2300'] == 'YES' else 0
    provides_anc = 1 if row['R1810'] == 'YES' else 0
    provides_testing = 1 if row['R2900'] == 'YES' else 0
    provides_oxygen = 1 if row["R1323"] == 'YES' else 0

    outpatients_ystrdy = float(row['tHFA_B034'])
    clincians_present = float(row['tHFA_B026'])
    clinicians_today = float(row['tHFA_B025'])

    # Attribute 1 (Integrated HTM with ANC)
    if provides_anc > 0:
        htm_anc_fxn = calculate_HTM_ANC(row)
        htm_anc_score = htm_anc_fxn['HTM-ANC']
    else:
        htm_anc_score = np.nan


    # Attribute 2 (Clincian presence) - Annex 1 definition is wrong (it is not the absence rate that is asked)
    if clinicians_today > 0 and clinicians_today != 999 and clincians_present != 999:
        presence = (clincians_present/clinicians_today)*100
    else: presence = np.nan 


    # Attribute 3 (Caseload)
    if clincians_present > 0 and outpatients_ystrdy != 999 and clincians_present != 999:
        caseload = (outpatients_ystrdy/clincians_present)*5
        if caseload >100:
            caseload = 100 
    else: caseload = np.nan

    

    # Attribute 4 (ANC dropout rate)
    anc_scores = []
    if provides_anc > 0:
        anc_count = sum(1 for col in ['RR_ANC_Begin_1', 'RR_ANC_Begin_2', 'RR_ANC_Begin_3', 'RR_ANC_Begin_4', 'RR_ANC_Begin_5'] if row[col] == 'YES')
        for i in range(1, anc_count+1):
            if math.isnan(row[f'Q13004_7_{i}']):
                anc_scores.append(np.nan)
            elif int(row[f'Q13004_7_{i}']) == 99 or int(row[f'Q13004_7_{i}']) == 98 :
                anc_scores.append(np.nan)
            elif int(row[f'Q13004_7_{i}']) > 3 :
                anc_scores.append(100) 
            else: anc_scores.append(0)

        anc_score = np.nanmean(anc_scores)

    else:
        anc_score = np.nan


    # Attribute 5 (DTP dropout rate)
    DPT_dropout = np.nan
    if provides_immu > 0:
        if float(row['tHFA_B032']) != 999 and float(row['tHFA_B032']) != 0 and float(row['tHFA_B033']) != 999:
            DPT_dropout = (float(row['tHFA_B032']) - float(row['tHFA_B033']))/float(row['tHFA_B032'])
            DPT_score = (1-DPT_dropout) * 100
            if DPT_score > 100: DPT_score = 100
            else: DPT_score = DPT_score

        else:
            DPT_score = np.nan
    
    else: 
        DPT_score = np.nan


    # Attribute 6 (treatment completion rate for new TB clients)
    if provides_TB > 0:
        tb_count = sum(1 for col in ['RR_TB_Begin_1', 'RR_TB_Begin_2', 'RR_TB_Begin_3', 'RR_TB_Begin_4', 'RR_TB_Begin_5'] if row[col] == 'YES')
        tb_trtmnt_scores = []
        
        for i in range(1, tb_count+1):
            tb_trtmnt = np.nan
            if row[f'Q13506_04_{i}'] == 'YES, CLIENT WAS CURED OR COMPLETED TREATMENT':
                tb_trtmnt = 100
            elif row[f'Q13506_04_{i}'] in ['CLIENT STILL ON TREATMENT', 'NOT DOCUMENTED',] :
                tb_trtmnt = np.nan
            else: 
                tb_trtmnt = 0

            tb_trtmnt_scores.append(tb_trtmnt)
        
        tb_trtmnt_score = np.nanmean(tb_trtmnt_scores)
    
    else: 
        tb_trtmnt_score = np.nan
        tb_trtmnt_scores = []

    # Attribute for ART 12 month retention
    if provides_ART > 0:   
        art_count = sum(1 for col in ['RR_ART_Begin_1', 'RR_ART_Begin_2', 'RR_ART_Begin_3', 'RR_ART_Begin_4', 'RR_ART_Begin_5'] if row[col] == 'YES')
        art_retnt_scores = []
        for i in range(1, art_count+1):
            art_retnt = np.nan
            if row[f'tHFAG002_{i}'] == 'NO': 
                art_retnt = np.nan
            elif row[f'tHFAG002_{i}'] == 'YES' and row[f'tHFAG005_{i}'] == 'YES':
                art_retnt = 100
            else:
                art_retnt = 0
            
            art_retnt_scores.append(art_retnt)
        art_retnt_score = np.nanmean(art_retnt_scores)
    else:
        art_retnt_score = np.nan
        art_retnt_scores = []
    

    # Aggregation of scores
    attribute_scores = [htm_anc_score,presence,caseload,anc_score,DPT_score,tb_trtmnt_score,art_retnt_score]
    comp_score = np.nanmean(attribute_scores)
    
    
    return pd.Series({
        'facility_composite': comp_score,
        'HTM-ANC': htm_anc_score,
        'presence_score': presence,
        'caseload': caseload,
        'ANC_score': anc_score,
        'DPT_score': DPT_score,
        'TB_treatment_completion': tb_trtmnt_score,
        'ART_retention': art_retnt_score,
        
        })


# see = test.apply(calculate_facility_composite, axis=1)
# see

In [84]:
# Function for calculation of CHW paid on time and in full

def calculate_CHW_paid(row):
    
    """

    RSSH/PP HRH-3: Percentage of community health workers remunerated on time and in-full
    
    Paid is calculated with 'tHFA_E020','tHFA_E021', 'tHFA_E022', 'tHFA_E023' and all need to be present to have the value of 100

    """
    chw_count = sum(1 for col in ['CHW_begin_1', 'CHW_begin_2', 'CHW_begin_3', 'CHW_begin_4', 'CHW_begin_5'] if row[col] == 'YES')

    chw_paid_scores = []

    paid_columns = ['tHFA_E020','tHFA_E021', 'tHFA_E022', 'tHFA_E023']

    for i in range(1, 6):
        chw_i_score = 0
        
        if row[f'tHFA_E002_{i}'] != 'YES':
            continue
        
        for col in paid_columns:
            interview_col = f'{col}_{i}'
            if row[interview_col] == 'YES': 
                chw_i_score = chw_i_score + 1
        chw_paid = 100 if chw_i_score == 4 else 0  
        chw_paid_scores.append(chw_paid)

    chw_paid_score = np.nanmean(chw_paid_scores)
    return pd.Series({
        'CHW_count': chw_count,
        'CHW_paid': chw_paid_score,
        'CHW_scores': chw_paid_scores  
        })

# see = df.apply(calculate_CHW_paid, axis=1)
# see

In [85]:
# Function for calculation of oxygen therapy

def calculate_oxygen(row):
    """
    RSSH/PP RCS-1: Percentage of health facilities able to provide oxygen therapy related services
    
    Using only oxygen currently available in this unit and oxygen be brought to this unit from a different unit

    """
    provides_oxygen = 'YES' if row["R1323"] == 'YES' else "NO"
    obs_options = ['OBSERVED']
    
    if provides_oxygen == 'YES':
       avl = 100 if any ([row ['R1324'] == 'YES', row['R1326'] == 'YES']) and all([row[f'R1325_0{i}A'] in obs_options for i in ['4','5','6']]) and all([row[f'R1325_0{i}B'] == 'YES' for i in ['4','5','6']]) else 0
       avo = 100 if all([row[f'R1322_09A'] in obs_options , row[f'R1322_09B'] == 'YES' ]) else 0
    else:
        avl = np.nan
        avo = np.nan
    
    score = np.nanmean([avl,avo])

    return pd.Series({
        'oxygen_score': score,
        'oxygen_avl': avl,
        'oxygen_avo': avo,

    })

# see = test.apply(calculate_oxygen, axis=1)
# see


In [86]:
# Function for the calculation of vacancy rate

def calculate_vacancy(row):
    """RSSH/PP HRH-1: Vacancy rate"""
    funded_posts = float(row['tHFA_B046'])
    unfilled_posts = float(row['tHFA_B047'])
    if (funded_posts == 0 and unfilled_posts == 0) or (funded_posts == 0):
        score = np.nan
    elif funded_posts != 999 and unfilled_posts != 999:
        score = ((unfilled_posts / funded_posts)) * 100 if unfilled_posts < funded_posts else 100
    else: score = np.nan 
    
    return   pd.Series({
        'vacancy_score': score,
        'all_funded_posts': funded_posts,
        'all_unfilled_posts': unfilled_posts,
    })

In [87]:
# Function for evaluating testing capacity
def calculate_testing_capacity(row):
    
    """

   Description
   RSSH/PP LAB-5: “Percentage of health facilities that have an appropriate set of diagnostics 
   for their healthcare facility level, based on adapted WHO model list of essential in vitro diagnostics (EDL 3)”


    This needs data for the EDL tests from the 
   
    """

    # Responses for facilities with labs
    test_facility_questions = [f'EDL_RESPONSE_0{i}' for i in range(1,10)]
    test_facility_questions.extend([f'EDL_RESPONSE_{i}' for i in range(10,28)])
    
    # Responses for facilities without labs 
    all_facility_questions = test_facility_questions + ['EDL_RESPONSE_28']
    remove = [f'EDL_RESPONSE_{i}' for i in range(10,19)] + [f'EDL_RESPONSE_0{i}' for i in range(8,10)] + [f'EDL_RESPONSE_{i}' for i in range(22,28)]
    remove_set = set(remove)
    non_test_facility_questions = [thing for thing in all_facility_questions if thing not in remove_set]
    

    test_capac_scores = []
    lab_score = np.nan
    no_lab_score = np.nan

    if row['R2900'] == 'YES, WITH LABORATORY IN THE FACILITY':
        for question in test_facility_questions:
            if row[question] == 'YES': test_capac_scores.append(100)
            # elif row[question] == 'NO': test_capac_scores.append(0)
            else: test_capac_scores.append(0)
        
        lab_score = np.nanmean(test_capac_scores) 

    elif row['R2900'] == 'YES, WITHOUT LABORATORY IN THE FACILITY':
        for question in non_test_facility_questions:
            if row[question] == 'YES': test_capac_scores.append(100)
            # elif row[question] == 'NO': test_capac_scores.append(0)
            else: test_capac_scores.append(0)
        
        no_lab_score = np.nanmean(test_capac_scores) 
        
    elif row['R2900'] == 'NO LABORATORY TESTS PERFORMED':
        test_capac_scores.append(np.nan)
    
    
    test_capac_score = np.nanmean(test_capac_scores)





    return pd.Series({
        'test-capac': test_capac_score,
        'test-capac_check': test_capac_scores,
        'with lab': lab_score,
        'without lab': no_lab_score,
        })

# # test = test.append(df.loc[['NGA_000170',
# # 'NGA_000167',
# # 'NGA_000163',
# # 'NGA_000045',
# # 'NGA_000124',
# # 'NGA_000516',
# # 'NGA_000311']
# # ])
# see = test.apply(calculate_testing_capacity, axis=1)
# see

In [88]:
# Function for guideline availability

def calculate_guideline_availability(row):
    
    """

   “Percentage of facilities with written and updated clinical guidelines for HIV, TB, malaria and/or 
   PHC (based on the services provided) developed by the national or sub-national government 
   (as appropriate by the country context)
   

    Returns a guideline availability score that is a np.nanmean of the scores for the relevant diseases 
    """

    provides_ART = 1 if row['R2312'] == 'YES' else 0
    provides_TB = 1 if row['R2400'] == 'YES' else 0
    provides_immu = 1 if row['R2100'] == 'YES' else 0
    provides_imci = 1 if row['R2000'] == 'YES' else 0
    provides_malaria = 1 if row['R1400'] == 'YES' else 0
    provides_hiv_test = 1 if row['R2300'] == 'YES' else 0
    provides_anc = 1 if row['R1810'] == 'YES' else 0
    provides_testing = 1 if row['R2900'] == 'YES' else 0
    provides_oxygen = 1 if row["R1323"] == 'YES' else 0

    positive_responses = ['YES, OBSERVED', 'YES, REPORTED, NOT SEEN' ]

    guidelines = [ provides_TB,
                provides_hiv_test, 
                  provides_ART,
                  provides_malaria,
                  provides_anc,
                  provides_immu,
                  provides_imci]
    
    guideline_cols = [ ['R2419_01','R2419_03'],
                        'R2308_01',
                      'R2315B_01',
                      
                      'R1406',
                      'R1817_01',
                      'R2119_01',
                      'R2005_01']
    guideline_scores = []
    if guidelines[0] > 0:
        if row[guideline_cols[0][0]] in positive_responses and row[guideline_cols[0][1]] in positive_responses:
            guideline_scores.append(100)
        else: guideline_scores.append(0)
    else: guideline_scores.append(np.nan)

    for i in range(1,len(guidelines)): 
        if guidelines[i] > 0 :
            if row[guideline_cols[i]] in positive_responses :
                guideline_scores.append(100)
            else: 
                guideline_scores.append(0)
        else: 
            guideline_scores.append(np.nan)
    
    
    guideline_score = np.nanmean(guideline_scores)

    return pd.Series({
        'guideline_score': guideline_score,
        # 'guideline_check': guideline_scores,
        'guide_HIV_test': guideline_scores[1],
        'guide_ART': guideline_scores[2],
        'guide_TB': guideline_scores[0],
        'guide_malaria': guideline_scores[3],
        'guide_anc': guideline_scores[4],
        'guide_immu': guideline_scores[5],
        'guide_imci': guideline_scores[6],
         })

# see = test.apply(calculate_guideline_availability, axis=1)
# see


In [89]:
# Function for  checking community-led mechanism 
# # Unclear how don't knows should be treated here
def calculate_CLM(row):
    
    """

   Indicator for CSS-3: “Percentage of health service delivery 
   sites with a community-led monitoring mechanism in place”

   Returns:
    0 - 100 based on number of strongly agree or agree responses

    """

    questions = ['tHFA_B021', 'tHFA_B022', 'tHFA_B023', 'tHFA_B024', 'tHFA_B035', 'tHFA_B045']
    agree_count = sum(1 for q in questions if row[q] in ['STRONGLY AGREE', 'AGREE'])
    score = (agree_count / len(questions)) * 100


    return pd.Series({
        'CLM_score': score,      
        })

### Service offered by facilities

In [90]:
# Function to show the services provided in each facility

def see_services(row):
    
    """

   Description


    """

    provides_ART = 1 if row['R2312'] == 'YES' else 0
    provides_TB = 1 if row['R2400'] == 'YES' else 0
    provides_immu = 1 if row['R2100'] == 'YES' else 0
    provides_imci = 1 if row['R2000'] == 'YES' else 0
    provides_malaria = 1 if row['R1400'] == 'YES' else 0
    provides_hiv_test = 1 if row['R2300'] == 'YES' else 0
    provides_anc = 1 if row['R1810'] == 'YES' else 0
    provides_testing = 1 if row['R2900'] == 'YES' else 0
    provides_HIV = 1 if provides_hiv_test + provides_ART > 0 else 0

    return pd.Series({
        'HIV': provides_HIV*100,
        'TB': provides_TB*100,
        'Malaria': provides_malaria*100,
        'ANC': provides_anc*100,
        'Immu': provides_immu*100,
    
        })

# services_df = facility_details_df

# services = df.apply(see_services, axis=1)
# services_df = pd.concat([services_df, services], axis=1)
# services_df

# services_df.to_csv('./tHFA-malawi-services_v1.0.csv')

## Function Implementation 

In [91]:
functions = [
    see_services,
    calculate_service_quality,
    calculate_ISS,
    calculate_HTM_ANC,
    calculate_system_CHW,
    calculate_facility_composite,
    calculate_CHW_paid,
    calculate_CLM,
    calculate_guideline_availability,
    calculate_oxygen,
    calculate_testing_capacity,
    calculate_vacancy,    
    ]

In [92]:
# test apply the calculation to each row

test_indicators_df = test_fac_df
for func in functions: 
    see = test.apply(func, axis=1)
    test_indicators_df = pd.concat([test_indicators_df, see], axis=1)

C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\866402996.py:222: RuntimeWarning: Mean of empty slice
  pc_i_score = np.nanmean(pc_i_scores) if len(pc_i_scores) > 0 else np.nan
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\866402996.py:225: RuntimeWarning: Mean of empty slice
  pc_score = np.nanmean(pc_scores) if len(pc_scores) > 0 else np.nan
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\866402996.py:228: RuntimeWarning: Mean of empty slice
  facility_score = np.nanmean([hw_score, pc_score])
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\866402996.py:246: RuntimeWarning: Mean of empty slice
  pc_attr_avg[f"PC_avg_{ques[i]}"] = np.nanmean(values)
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\2005688502.py:105: RuntimeWarning: Mean of empty slice
  'fac_tech': np.nanmean(fac_tech),
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\896996215.py:85: RuntimeWarning: Mean of empty slice
  ART_access_score = np.nanmean(ART_access_scores) i

In [93]:
# FULL Apply the calculation to each row
indicators_df = facility_details_df

for func in functions: 
    see = df.apply(func, axis=1)
    indicators_df = pd.concat([indicators_df, see], axis=1)

indicators_df

C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\866402996.py:222: RuntimeWarning: Mean of empty slice
  pc_i_score = np.nanmean(pc_i_scores) if len(pc_i_scores) > 0 else np.nan
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\866402996.py:225: RuntimeWarning: Mean of empty slice
  pc_score = np.nanmean(pc_scores) if len(pc_scores) > 0 else np.nan
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\866402996.py:228: RuntimeWarning: Mean of empty slice
  facility_score = np.nanmean([hw_score, pc_score])
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\866402996.py:246: RuntimeWarning: Mean of empty slice
  pc_attr_avg[f"PC_avg_{ques[i]}"] = np.nanmean(values)
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\2005688502.py:105: RuntimeWarning: Mean of empty slice
  'fac_tech': np.nanmean(fac_tech),
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\2005688502.py:106: RuntimeWarning: Mean of empty slice
  'fac_stats': np.nanmean(fac_stats),
C:\Users\Seu

,Q102,Q105,Q106,Q105_a,Q113,Q113_A,Q116,Q116_A,Q117,Q118,...,oxygen_score,oxygen_avl,oxygen_avo,test-capac,test-capac_check,with lab,without lab,vacancy_score,all_funded_posts,all_unfilled_posts
Q100,,,,,,,,,,,,,,,,,,,,,
GMB_000104,Chamboi Kunda Health Post,Upper River,Chamboi Kunda,PRIMARY,COMMUNITY CLINIC,NaN,MINISTRY OF HEALTH (GOVERNMENT),NaN,RURAL,OUTPATIENT ONLY,...,NaN,NaN,NaN,7.407407,"[0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",7.407407,NaN,0.000000,2.0,0.0
GMB_000110,Foday Kunda Health Centre,Upper River,Foday Kunda,SECONDARY,HEALTH CENTRE,NaN,MINISTRY OF HEALTH (GOVERNMENT),NaN,RURAL,BOTH OUT AND INPATIENT,...,NaN,NaN,NaN,9.090909,"[0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0]",NaN,9.090909,20.000000,5.0,1.0
GMB_000105,Changally Lang Kaddy,Upper River,Changally Lang Kaddy,PRIMARY,HEALTH CENTRE,NaN,MINISTRY OF HEALTH (GOVERNMENT),NaN,RURAL,OUTPATIENT ONLY,...,50.0,0.0,100.0,9.090909,"[0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0]",NaN,9.090909,33.333333,3.0,1.0
GMB_000114,Kuwunkuba Health Post,Upper River,Kuwonkuba,PRIMARY,HEALTH POST,NaN,MINISTRY OF HEALTH (GOVERNMENT),NaN,RURAL,OUTPATIENT ONLY,...,NaN,NaN,NaN,NaN,[nan],NaN,NaN,NaN,999.0,999.0
GMB_000100,Bajakunda Health Centre,Upper River,Baja Kunda,SECONDARY,HEALTH CENTRE,NaN,MINISTRY OF HEALTH (GOVERNMENT),NaN,RURAL,BOTH OUT AND INPATIENT,...,0.0,0.0,0.0,44.444444,"[100, 0, 0, 100, 100, 100, 100, 100, 0, 0, 0, ...",44.444444,NaN,30.555556,72.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GMB_000025,SOS Clinic,Kanifing,Bakoteh,SECONDARY,CLINIC/DISPENSARY,NaN,PRIVATE-FOR-PROFIT,NaN,URBAN,OUTPATIENT ONLY,...,100.0,100.0,100.0,37.037037,"[100, 0, 0, 0, 100, 100, 0, 100, 0, 0, 0, 100,...",37.037037,NaN,18.604651,43.0,8.0
GMB_000027,Sharab Medical Clinic,Kanifing,Kanifing,SECONDARY,CLINIC/DISPENSARY,NaN,PRIVATE-FOR-PROFIT,NaN,URBAN,BOTH OUT AND INPATIENT,...,100.0,100.0,100.0,55.555556,"[100, 0, 0, 0, 100, 0, 0, 100, 100, 100, 100, ...",55.555556,NaN,0.000000,127.0,0.0
GMB_000087,Medina Serign Mass Clinic,North Bank,Medina Serign Mass,SECONDARY,COMMUNITY CLINIC,NaN,MINISTRY OF HEALTH (GOVERNMENT),NaN,RURAL,OUTPATIENT ONLY,...,50.0,100.0,0.0,9.090909,"[0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0]",NaN,9.090909,55.000000,20.0,11.0


## Aggregation

In [94]:
# Step 7: Function to aggregate indicators by region and facility type
def aggregate_indicators(df, group_by_columns):
    out  = pd.concat([df.groupby(group_by_columns).size(), df.groupby(group_by_columns).mean()], axis=1)
    out = out.rename(columns={0:'Facility count'})
    return  out

# Example usage:
aggregated_by_type = aggregate_indicators(indicators_df, ['tHFA_A001','Q117'])

In [95]:
aggregated_by_ty = aggregate_indicators(indicators_df, ['tHFA_A001'])
for idx in aggregated_by_ty.index:
    see = pd.DataFrame()
    see = aggregated_by_ty.loc[idx]
    see.name = (f"{idx}","General")
    aggregated_by_type = aggregated_by_type.append(see)

aggregated_by_location = aggregate_indicators(indicators_df, ['Q117'])
for idx in aggregated_by_location.index:
    see = pd.DataFrame()
    see = aggregated_by_location.loc[idx]
    see.name = (f"{idx}","General")
    aggregated_by_type = aggregated_by_type.append(see)

In [96]:
mean = pd.Series()
mean.at['Facility count'] = len(indicators_df)
if code in ["UGA"]:
    sharp = indicators_df.mean()[3:]
elif code in ['GMB','IDN','COG']:
    sharp = indicators_df.mean()[2:]
else: sharp = indicators_df.mean()[1:]
mean  = pd.concat([mean,sharp])
mean.name = ('COUNTRY', 'AVERAGE')
mean

C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\2073807711.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mean = pd.Series()
C:\Users\Seun Osonuga\AppData\Local\Temp\ipykernel_5984\2073807711.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  sharp = indicators_df.mean()[2:]


Facility count        120.000000
HIV                    60.000000
TB                     33.333333
Malaria                98.333333
ANC                    66.666667
                         ...    
with lab               28.690662
without lab            13.286713
vacancy_score          41.646163
all_funded_posts      159.375000
all_unfilled_posts    153.275000
Name: (COUNTRY, AVERAGE), Length: 66, dtype: float64

In [97]:
aggregated_by_type = aggregated_by_type.append(mean)
aggregated_by_type = aggregated_by_type.sort_index()
aggregated_by_type

Facility count         HIV          TB     Malaria  \
tHFA_A001 Q117                                                            
COUNTRY   AVERAGE             120.0   60.000000   33.333333   98.333333   
PERIURBAN General               7.0   57.142857   42.857143  100.000000   
PRIMARY   General              72.0   44.444444   18.055556   97.222222   
          PERIURBAN             4.0   50.000000   25.000000  100.000000   
          RURAL                57.0   42.105263   19.298246   96.491228   
          URBAN                11.0   54.545455    9.090909  100.000000   
RURAL     General              87.0   54.022989   32.183908   97.701149   
SECONDARY General              39.0   84.615385   51.282051  100.000000   
          PERIURBAN             1.0  100.000000  100.000000  100.000000   
          RURAL                27.0   77.777778   55.555556  100.000000   
          URBAN                11.0  100.000000   36.363636  100.000000   
TERTIARY  General               9.0   77.777778   77.777778  100.000000   
          PERIURBAN             2.0   50.000000   50.000000  100.000000   
          RURAL                 3.0   66.666667   66.666667  100.000000   
          URBAN                 4.0  100.000000  100.000000  100.000000   
URBAN     General              26.0   80.769231   34.615385  100.000000   

                            ANC        Immu  Quality_Score  \
tHFA_A001 Q117                                               
COUNTRY   AVERAGE     66.666667   64.166667      71.067377   
PERIURBAN General     85.714286   71.428571      66.048796   
PRIMARY   General     55.555556   51.388889      71.828261   
          PERIURBAN  100.000000   75.000000      64.501068   
          RURAL       47.368421   42.105263      71.804048   
          URBAN       81.818182   90.909091      74.600551   
RURAL     General     57.471264   54.022989      70.509228   
SECONDARY General     84.615385   84.615385      70.143378   
          PERIURBAN  100.000000  100.000000      62.809524   
          RURAL       77.777778   77.777778      68.038537   
          URBAN      100.000000  100.000000      75.785173   
TERTIARY  General     77.777778   77.777778      69.557981   
          PERIURBAN   50.000000   50.000000      70.763889   
          RURAL       66.666667   66.666667      70.773148   
          URBAN      100.000000  100.000000      68.043651   
URBAN     General     92.307692   96.153846      74.092983   

                     HW_Competence_Score  ANC_score  ART_score  ...  \
tHFA_A001 Q117                                                  ...   
COUNTRY   AVERAGE              65.552326  69.373016  45.416667  ...   
PERIURBAN General              56.366300  58.000000  41.111111  ...   
PRIMARY   General              67.198140  66.727920  50.000000  ...   
          PERIURBAN            55.807692  51.000000  50.000000  ...   
          RURAL                68.001367  68.397493  50.000000  ...   
          URBAN                70.082071  68.957071        NaN  ...   
RURAL     General              65.634506  70.521313  42.291667  ...   
SECONDARY General              63.792306  73.096774  41.842105  ...   
          PERIURBAN            54.666667  64.000000  50.000000  ...   
          RURAL                62.528091  73.809524  35.769231  ...   
          URBAN                68.037037  72.444444  56.000000  ...   
TERTIARY  General              64.583333  67.619048  48.571429  ...   
          PERIURBAN            58.333333  80.000000  23.333333  ...   
          RURAL                68.222222  64.666667  50.000000  ...   
          URBAN                65.888889  66.000000  54.166667  ...   
URBAN     General              68.406926  69.888408  55.185185  ...   

                     guide_imci  oxygen_score  oxygen_avl  oxygen_avo  \
tHFA_A001 Q117                                                          
COUNTRY   AVERAGE     87.500000     72.142857   57.142857   87.142857   
PERIURBAN General     85.714286     60.000000   40.0000

## Saving 

In [98]:
# Step 8: Save outputs to CSV files

aggregated_by_type.to_csv(pivot_out)
indicators_df.to_csv(indicators_out)

print(f"Data processing completed and files saved to {pivot_out} and {indicators_out}")

Data processing completed and files saved to ./output/GMB/gambia_pivot_v2.7.csv and ./output/GMB/tHFA-gambia-analysis_v2.7.csv


In [99]:

# metadata_df = pd.DataFrame(metadata)
# metadata_df.to_csv('/mnt/data/metadata.csv', index=False)
# survey_details_df.to_csv('/mnt/data/survey_details.csv', index=False)
# indicators_df.to_csv('/mnt/data/indicators.csv', index=False)
# aggregated_by_region.to_csv('/mnt/data/aggregated_by_region.csv', index=False)
# aggregated_by_facility_type.to_csv('/mnt/data/aggregated_by_facility_type.csv', index=False)

# 
